In [2]:
from gatenlp import Document
from gatenlp.gateworker import GateWorker

import os
os.environ["GATE_HOME"] = r"D:/Study/Setup/gatenlp"

# Create a document
doc = Document("Barack Obama was the 44th President of the United States.")

# you can specify auth_token explicitly
gw = GateWorker(start=True, auth_token="mySecretToken")

# or omit it, then a random one is generated
gw = GateWorker(start=True)  
print("Auth token is:", gw._auth_token)


2025-10-02 14:38:11,599|INFO|gatenlp.gateworker.gateworker|Port 25333 is already in use
Trying to start GATE Worker on port=25334 host=127.0.0.1 log=false keep=false
Process id is 20032
PythonWorkerRunner.java: starting server with 25334/127.0.0.1/mySecretToken/false
2025-10-02 14:38:17,509|INFO|gatenlp.gateworker.gateworker|Port 25333 is already in use
2025-10-02 14:38:17,511|INFO|gatenlp.gateworker.gateworker|Port 25334 is already in use
Trying to start GATE Worker on port=25335 host=127.0.0.1 log=false keep=false
Process id is 19640


Auth token is: _XAw3yoyAv9g7NDrDMLm_860-c0


PythonWorkerRunner.java: starting server with 25335/127.0.0.1/_XAw3yoyAv9g7NDrDMLm_860-c0/false
